In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import os, os.path 
import numpy 
import pickle
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

"""Change to the data folder"""
train_path = "../data/new_train"
val_path = '../data/new_val_in/new_val_in'

In [2]:
class ArgoverseDataset(Dataset):
    
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self): #len(val_dataset)
        return len(self.pkl_list)
    
    def __getitem__(self, idx): #val_dataset[0]
        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        if self.transform:
            data = self.transform(data)
        return data

In [3]:
init_dataset = ArgoverseDataset(data_path=train_path) 
val_dataset  = ArgoverseDataset(data_path=val_path) 
print(len(init_dataset), len(val_dataset))

205942 3200


In [4]:
lengths = [int(len(init_dataset)*0.7) + 1, int(len(init_dataset)*0.3)]
#lengths = [int(len(init_dataset)*0.2) + 1, int(len(init_dataset)*0.5), int(len(init_dataset)*0.3)]

print(lengths)
train_dataset, test_dataset = random_split(init_dataset, lengths)
#print(len(small_train_dataset), len(test_dataset), len(val_dataset))

[144160, 61782]


In [8]:
batch_sz = 14
def my_collate(batch): #[scene['p_in'][scene['car_mask'].reshape(-1) == 1]]
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp = torch.FloatTensor(inp) #LongTensor
    out = torch.FloatTensor(out) #LongTensor
    return [inp, out]
train_loader = DataLoader(train_dataset, batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=0)

In [15]:
class base_model(nn.Module):
    def __init__(self, input_size = 240):
        super().__init__()
        self.hidden_layer_size = 3
        self.hidden_dim = 2048

        self.lstm = nn.LSTM(input_size, self.hidden_dim, num_layers = self.hidden_layer_size, batch_first = True)

        self.linear = nn.Linear(self.hidden_dim, 240, 1)


    def forward(self, x):
        x,_ = self.lstm(x)
        print(x.shape)
        #x = x.transpose(1, 2)
        x = self.linear(x)
        x = x.transpose(1, 2)
        print(x.shape)
        
        return x
    
    def forward_test(self, x, steps = 30):
        fut = []
        h = torch.zeros((self.hidden_layer_size, len(x), self.hidden_dimen)).cuda()
        c = torch.zeros((self.hidden_layer_size, len(x), self.hidden_dimen)).cuda()
        for num_iter in steps:
            x, (h, c) = self.lstm(x, (h, c))
            x = x[:, -1:]
            x = x.transpose(1, 2)
            x = self.linear(x)
            x = x.transpose(1, 2)
            fut.append(x)
        fut = torch.cat(fut, 1)
        return fut

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = base_model().cuda()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

cuda
base_model(
  (lstm): LSTM(240, 2048, num_layers=3, batch_first=True)
  (linear): Linear(in_features=2048, out_features=240, bias=True)
)


In [17]:
model.train()
epochs = 20
loss_ema = -1
loss_ema_test = -1

for i in range(epochs):
    print('epoch', i)
    #iterator = tqdm(train_loader, total=int(len(train_loader)))
    for i_batch, sample_batch in enumerate(train_loader):
        seq, labels = sample_batch
        seq = seq.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        full = torch.cat([seq, labels], 2).transpose(1, 2).reshape((-1, 49, 240)).float()
        
        y_pred = model(full[:, :-1])[:, -30:]
        y_pred = y_pred.reshape((-1, 30, 60, 4)).transpose(1, 2)
        
        loss = loss_function(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if loss_ema < 0:
            loss_ema = loss
        loss_ema = loss_ema*0.99 + loss * 0.01
        
        with torch.no_grad():
            y_pred_test = model.forward_test(seq.transpose(1,2).reshape(-1, 19, 240))
            y_pred_test = y_full_pred.reshape((-1, 30, 60, 4)).transpose(1, 2)
            loss_test= loss_function(y_pred_test, label)
            if loss_ema_test < 0:
                loss_ema_test = loss_test
            loss_ema_test = loss_ema_test*0.99 + loss_test * 0.01
            
        print("Full loss", epoch, i_batch, loss_ema.item(), loss.item())
        print("Test loss", epoch, i_batch, loss_ema_test.item(), loss_test.item())

epoch 0
torch.Size([14, 48, 2048])
torch.Size([14, 48, 240])
torch.Size([14, 240, 48])


RuntimeError: shape '[-1, 30, 60, 4]' is invalid for input of size 20160

In [ ]:
torch.save(model.state_dict(), "trial1.pt")